<a href="https://colab.research.google.com/github/zuki2610/bigdata/blob/main/Entregaip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar PySpark en el entorno de Google Colab
!pip install pyspark

# Importar PySpark y otras librerías necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, min, countDistinct

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:

# Ruta a los archivos en la carpeta "Datos" de tu Google Drive
ruta_segmentacion = "/content/drive/My Drive/Datos/Segmentacion.csv"
ruta_precios = "/content/drive/My Drive/Datos/Precios.csv"

# Crear una sesión de Spark
spark = SparkSession.builder.appName("PySpark Example").getOrCreate()

# Cargar los archivos CSV en DataFrames de PySpark
segmentacion_df = spark.read.csv(ruta_segmentacion, header=True, inferSchema=True)
precios_df = spark.read.csv(ruta_precios, header=True, inferSchema=True)

# Mostrar una vista previa de los datos
print("Vista previa de la segmentación:")
segmentacion_df.show(5)
print("Vista previa de los precios:")
precios_df.show(5)

# *1. Validar cuántos registros tienen ambos archivos*
segmentacion_count = segmentacion_df.count()
precios_count = precios_df.count()

print(f"Registros en segmentación: {segmentacion_count}")
print(f"Registros en precios: {precios_count}")

Vista previa de la segmentación:
+--------+--------+
| RUT_CLI|SEGMENTO|
+--------+--------+
|96635170|       1|
|76256513|       1|
|77108943|       1|
|92975000|       1|
|89201400|       1|
+--------+--------+
only showing top 5 rows

Vista previa de los precios:
+-----+---+------+------------------+
|GROUP|FAM|SUBFAM|             VALUE|
+-----+---+------+------------------+
|    0|  1|     1|0.2350543478260869|
|    0|  1|     1| 0.198444661032879|
|    0|  1|     1|0.2919982088695311|
|    0|  1|     1|0.2272116796417313|
|    0|  1|     1|0.2518425219231205|
+-----+---+------+------------------+
only showing top 5 rows

Registros en segmentación: 126104
Registros en precios: 11018159


In [7]:
# *2. Realizar una partición de la matriz precio por familia-subfamilia*
precios_partitioned = precios_df.repartition("fam", "subfam")
print("Matriz de precios particionada por familia y subfamilia.")
print(precios_partitioned.show())

Matriz de precios particionada por familia y subfamilia.
+-----+---+------+------------------+
|GROUP|FAM|SUBFAM|             VALUE|
+-----+---+------+------------------+
|    0| 10|     2|0.3857003916848538|
|    0| 10|     2|0.3677940968114483|
|    0| 10|     2|0.3901394142121732|
|    0| 10|     2|0.3030408779958913|
|    0| 10|     2|0.3586941963298005|
|    0| 10|     2|0.3612035640135574|
|    0| 10|     2|0.3106493323990148|
|    0| 10|     2|0.1715160796324655|
|    0| 10|     2|0.3618363058629711|
|    0| 10|     2|0.1961770267341639|
|    0| 10|     2|0.4078984485190409|
|    0| 10|     2|0.3257364115216078|
|    0| 10|     2|0.3437300811423489|
|    0| 10|     2|0.3431603096604355|
|    0| 10|     2|0.2839106891413789|
|    0| 10|     2|0.3891248934304149|
|    0| 10|     2|0.3099415204678363|
|    0| 10|     2|0.3975318912922906|
|    0| 10|     2|0.3833524561218172|
|    0| 10|     2|0.3093344534992925|
+-----+---+------+------------------+
only showing top 20 rows

None


In [3]:
#3 Validar que los valores (márgenes) estén entre 0.1 y 1
valid_margins = precios_df.filter((col("value") >= 0.1) & (col("value") <= 1))

# Contar cuántos registros tienen márgenes válidos
valid_margins_count = valid_margins.count()

# Contar el total de registros
total_count = precios_df.count()

print(f"Registros con márgenes válidos (0.1 a 1): {valid_margins_count}")
print(f"Total de registros en el DataFrame: {total_count}")
print(f"Porcentaje de márgenes válidos: {100 * valid_margins_count / total_count:.2f}%")

Registros con márgenes válidos (0.1 a 1): 11018159
Total de registros en el DataFrame: 11018159
Porcentaje de márgenes válidos: 100.00%


In [7]:
from pyspark.sql.functions import col, countDistinct, avg

'''
# Cargar los archivos CSV en DataFrames de PySpark
segmentacion_df = spark.read.csv(ruta_segmentacion, header=True, inferSchema=True)
precios_df = spark.read.csv(ruta_precios, header=True, inferSchema=True)

# Mostrar una vista previa de los datos
print("Vista previa de la segmentación:")
segmentacion_df.show(5)
print("Vista previa de los precios:")
precios_df.show(5)
'''
# Realizar la unión entre los DataFrames en GROUP y SEGMENTO
segmentacion_precios_df = precios_df.join(
    segmentacion_df,
    segmentacion_df["SEGMENTO"] == precios_df["GROUP"],
    "inner"
)

# Contar clientes únicos (RUT_CLI) por grupo (GROUP)
clientes_por_grupo = segmentacion_precios_df.groupBy("GROUP").agg(
    countDistinct("RUT_CLI").alias("clientes_unicos")
)

# Calcular el promedio de clientes únicos por grupo
promedio_clientes_por_grupo = clientes_por_grupo.agg(
    avg("clientes_unicos").alias("promedio_clientes")
).collect()[0]["promedio_clientes"]

print(f"Promedio de clientes únicos por grupo: {promedio_clientes_por_grupo}")


Promedio de clientes únicos por grupo: 26.910798122065728


In [4]:
from pyspark.sql.functions import col, max, avg, min

# Realizar la unión entre los DataFrames en SEGMENTO y GROUP
segmentacion_precios_df = segmentacion_df.join(
    precios_df,
    segmentacion_df["SEGMENTO"] == precios_df["GROUP"],
    "inner"
)

# Filtrar los registros para el RUT_CLI = 66666666
rut_66666666_df = segmentacion_precios_df.filter(col("RUT_CLI") == 66666666)

# Calcular márgenes (valores) máximos, promedios y mínimos
margen_estadisticas = rut_66666666_df.agg(
    max("VALUE").alias("margen_maximo"),
    avg("VALUE").alias("margen_promedio"),
    min("VALUE").alias("margen_minimo")
).collect()

# Extraer los resultados
margen_maximo = margen_estadisticas[0]["margen_maximo"]
margen_promedio = margen_estadisticas[0]["margen_promedio"]
margen_minimo = margen_estadisticas[0]["margen_minimo"]

# Mostrar resultados
print(f"Márgenes para el RUT_CLI 66666666:")
print(f"Máximo: {margen_maximo}")
print(f"Promedio: {margen_promedio}")
print(f"Mínimo: {margen_minimo}")

Márgenes para el RUT_CLI 66666666:
Máximo: 0.7415512051396618
Promedio: 0.39309233080261313
Mínimo: 0.2106912324480014
